In [25]:
import pandas as pd
import numpy as np
import os
import sys
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [ ]:

class BaseSignalStrategy(bt.Strategy):
    params = (('record_signals', True),)

    def __init__(self):
        self.signal = 0  # 当前信号
        self.signals = []  # 记录历史信号

    def next(self):
        if self.record_signals:
            self.signals.append({
                'date': self.data.datetime.date(0),
                'signal': self.signal
            })

    def get_signals_df(self):
        return pd.DataFrame(self.signals).set_index('date')



In [ ]:

# --------------------------
# 独立指标策略（按需扩展）
# --------------------------
class RSIStrategy(BaseSignalStrategy):
    params = (('rsi_period', 14), ('oversold', 30), ('overbought', 70))

    def __init__(self):
        super().__init__()
        self.rsi = bt.indicators.RSI(self.data.close, period=self.p.rsi_period)

    def next(self):
        if self.rsi < self.p.oversold:
            self.signal = 1  # 买入信号
        elif self.rsi > self.p.overbought:
            self.signal = -1  # 卖出信号
        else:
            self.signal = 0
        super().next()



In [ ]:

class CCIStrategy(BaseSignalStrategy):
    params = (('cci_period', 20), ('oversold', -100), ('overbought', 100))

    def __init__(self):
        super().__init__()
        self.cci = bt.indicators.CCI(self.data, period=self.p.cci_period)

    def next(self):
        if self.cci < self.p.oversold:
            self.signal = 1
        elif self.cci > self.p.overbought:
            self.signal = -1
        else:
            self.signal = 0
        super().next()


In [ ]:


class MACDStrategy(BaseSignalStrategy):
    params = (('fast', 12), ('slow', 26), ('signal_period', 9))

    def __init__(self):
        super().__init__()
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.p.fast,
            period_me2=self.p.slow,
            period_signal=self.p.signal_period
        )
        self.crossover = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)

    def next(self):
        if self.crossover > 0:
            self.signal = 1  # 金叉买入
        elif self.crossover < 0:
            self.signal = -1  # 死叉卖出
        else:
            self.signal = 0
        super().next()



In [ ]:
class StochasticStrategy(bt.Strategy):
    params = (
        ('k_period', 14),    # %K计算周期
        ('d_period', 3),     # %D计算周期（%K的SMA）
        ('overbought', 80),  # 超买阈值
        ('oversold', 20),    # 超卖阈值
        ('use_trend_filter', True),  # 是否使用趋势过滤
    )

    def __init__(self):
        # 计算指标
        self.stoch = bt.indicators.Stochastic(
            self.data,
            period=self.p.k_period,
            period_dfast=self.p.d_period,
            upperband=self.p.overbought,
            lowerband=self.p.oversold
        )
        self.crossover = bt.indicators.CrossOver(self.stoch.lines.percK, self.stoch.lines.percD)
        
        # 趋势过滤（可选）
        if self.p.use_trend_filter:
            self.sma200 = bt.indicators.SMA(self.data.close, period=200)

    def next(self):
        # 生成信号（1=买入，-1=卖出，0=无信号）
        signal = 0
        
        # 多头条件
        if (self.stoch.lines.percK[0] < self.p.oversold and 
            self.crossover > 0 and 
            (not self.p.use_trend_filter or self.data.close[0] > self.sma200[0])):
            signal = 1
        
        # 空头条件
        elif (self.stoch.lines.percK[0] > self.p.overbought and 
              self.crossover < 0 and 
              (not self.p.use_trend_filter or self.data.close[0] < self.sma200[0])):
            signal = -1
        
        # 记录信号（用于后续机器学习组合）
        self.signal = signal

In [ ]:

class MOMStrategy(BaseSignalStrategy):
    params = (('mom_period', 10), ('threshold', 0))

    def __init__(self):
        super().__init__()
        self.mom = bt.indicators.Momentum(self.data.close, period=self.p.mom_period)

    def next(self):
        if self.mom > self.p.threshold:
            self.signal = 1  # 动量向上
        elif self.mom < -self.p.threshold:
            self.signal = -1  # 动量向下
        else:
            self.signal = 0
        super().next()